# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# !pip install python-dotenv
import os
from dotenv import load_dotenv
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the API key from the environment variable and store as Python variable
geoapify_key = os.getenv("geoapify_key")
type(geoapify_key)
#The instructor gave me this code

str

In [2]:
!pip show hvplot

Name: hvplot
Version: 0.9.2
Summary: A high-level plotting API for the PyData ecosystem built on HoloViews.
Home-page: https://hvplot.holoviz.org
Author: Philipp Rudiger
Author-email: developers@pyviz.org
License: BSD
Location: C:\Users\HP\anaconda3\Lib\site-packages
Requires: bokeh, colorcet, holoviews, numpy, packaging, pandas, panel, param
Required-by: 


In [3]:
!pip show geoviews

Name: geoviews
Version: 1.11.1
Summary: GeoViews is a Python library that makes it easy to explore and visualize geographical, meteorological, and oceanographic datasets, such as those used in weather, climate, and remote sensing research.
Home-page: https://geoviews.org
Author: 
Author-email: 
License: BSD 3-Clause
Location: C:\Users\HP\anaconda3\Lib\site-packages
Requires: bokeh, cartopy, holoviews, numpy, packaging, panel, param, pyproj, shapely, xyzservices
Required-by: 


In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pisco,-13.7000,-76.2167,295.18,83,0,3.09,PE,1712802251
1,1,aasiaat,68.7098,-52.8699,263.82,82,100,1.62,GL,1712802420
2,2,grytviken,-54.2811,-36.5092,274.33,95,32,6.53,GS,1712802420
3,3,tavda,58.0425,65.2726,271.87,88,90,2.24,RU,1712802420
4,4,albany,42.6001,-73.9662,285.14,95,100,1.28,US,1712802138


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    size = "Humidity",
    hover_color = ["City"]
)

# Display the map
cities_map

#I started this code, but the TA wrote most of it for me

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
cities_filtered = pd.DataFrame(city_data_df.loc[(city_data_df["Wind Speed"]<4.5) & (city_data_df["Max Temp"]<300)])

# Drop any rows with null values
cities_filtered = cities_filtered.dropna()

# Display sample data
cities_filtered.head()

#The TA helped me with this, but I think I wrote most of it and drove it's structure.

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,pisco,-13.7000,-76.2167,295.18,83,0,3.09,PE,1712802251
1,1,aasiaat,68.7098,-52.8699,263.82,82,100,1.62,GL,1712802420
3,3,tavda,58.0425,65.2726,271.87,88,90,2.24,RU,1712802420
4,4,albany,42.6001,-73.9662,285.14,95,100,1.28,US,1712802138
10,10,mount gambier,-37.8333,140.7667,288.04,59,100,2.97,AU,1712802421


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
import warnings
warnings.filterwarnings("ignore")
#The TA wrote this

In [9]:
cities_filtered[cities_filtered.City == "waitangi"]
#The TA wrote this

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cities_filtered[["City","Country", "Lat","Lng","Humidity"]]  #the TA wrote this line
# hotel_df = hotel_df.T.set_index("City")
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""  #The TA told me to write this line

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,pisco,PE,-13.7000,-76.2167,83,
1,aasiaat,GL,68.7098,-52.8699,82,
3,tavda,RU,58.0425,65.2726,88,
4,albany,US,42.6001,-73.9662,95,
10,mount gambier,AU,-37.8333,140.7667,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit" : 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = row["Lng"]
    latitiude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitiude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitiude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(10)

# I framed out about 5 lines of code, but then the TA rewrote the rest of this for me while walking me through it.
# It didn't work because the params were not correct, but Khaled Karman trouble-shooted that and got it fixed for me.

Starting hotel search
pisco - nearest hotel: La Portada
aasiaat - nearest hotel: SØMA
tavda - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
mount gambier - nearest hotel: The Old Mount Gambier Gaol
iqaluit - nearest hotel: Frobisher Inn
ust-nera - nearest hotel: гостиница "Солнечная"
krapivinskiy - nearest hotel: No hotel found
susuman - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
dryden - nearest hotel: Holiday Inn Express
lompoc - nearest hotel: Red Roof Inn Lompoc
tiksi - nearest hotel: Арктика
hammerfest - nearest hotel: Thon Hotel Hammerfest
leshan - nearest hotel: Jinye Express Hotel
chongwe - nearest hotel: No hotel found
ushtobe - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
port elizabeth - nearest hotel: Waterford Hotel
shindand - nearest hotel: No hotel found
invercargill 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,pisco,PE,-13.7000,-76.2167,83,La Portada
1,aasiaat,GL,68.7098,-52.8699,82,SØMA
3,tavda,RU,58.0425,65.2726,88,No hotel found
4,albany,US,42.6001,-73.9662,95,No hotel found
10,mount gambier,AU,-37.8333,140.7667,59,The Old Mount Gambier Gaol
11,iqaluit,CA,63.7506,-68.5145,86,Frobisher Inn
14,ust-nera,RU,64.5667,143.2000,84,"гостиница ""Солнечная"""
15,krapivinskiy,RU,54.9992,86.8133,76,No hotel found
16,susuman,RU,62.7833,148.1667,77,No hotel found
17,edinburgh of the seven seas,SH,-37.0676,-12.3116,80,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
# Configure the map plot
hotels_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1,
    hover_cols = ["Hotel Name", "City"]
)

# Display the map
hotels_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City)